# Catenable Deques

### Overview
|         instance         | persistence | amortization | empty | isEmpty |  cons  |  snoc  |     ++     | head |  tail | last |  init |
|:------------------------:|:-----------:|:------------:|:-----:|:-------:|:------:|:------:|:----------:|:----:|:-----:|:----:|:-----:|
|  Simple Catenable Deque  |  persistent |      yes     |  O(1) |   O(1)  | O(1)** | O(1)** | O(log(n))* | O(1) | O(1)* | O(1) | O(1)* |
| Implicit Catenable Deque |  persistent |      yes     |  O(1) |   O(1)  | O(1)** | O(1)** |    O(1)*   | O(1) | O(1)* | O(1) | O(1)* |

** amortized time*

*** amortized or worst-case time (depends on the underlying deque instance)*

Note that both instances assume that there is an underlying deque which supports all operations in $O(1)$ amortized or worst-case time.

In [ ]:
class Deque q where

    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- Insert, inspect and remove the rear element
    snoc :: q a -> a -> q a
    last :: q a -> a
    init :: q a -> q a

class Deque d => CatenableDeque d where

    -- | Concatenate two deques together
    (++) :: d a -> d a -> d a

class Sized d where

    -- | Retrieve the size of a data structure 'd'
    size :: d a -> Int

## Simple Catenable Deque
`SimpleCatDeque` is a *structural abstraction* of standard `Deque`s that implements the *implicit recursive slowdown* to provide a concatenation operator that runs in $O(log(n))$ amortized time.

In the implementation and discussion below we assume that there is an existing `Deque` instance that with $O(1)$ operations (either worst-case or amortized). For example both `BankersDeque` and `RealTimeDeque` satisfy this assumption.

### Representation
`SimpleCatDeque d` structurally abstracts a *primitive* `Deque d` and as such forms a non-uniformly recursive structure. Moreover, since it follows the *implicit recursive slowdown*, this deque has two type constructors:
 1. `Shallow` simply holds a primitive deque `d`
 1. `Deep` has three parts: the front - a primitive deque, middle - `SimpleCatDeque` with `d` as elements and rear - also a primitive deque
 
There is an addition invariant imposed on the `Deep` deque which states that both the front and rear primitive deques must contain at least 2 element (otherwise the deque must be downgraded to a `Shallow` one).

### Operations
Starting with the set of simple operations: `cons`, `snoc`, `head` and `last` all simply propagate to the respective operation on the underlying primitive `Deque`. That is either from the `Shallow` type or in case of a `Deep` deque according to the operation to the front or rear part.

`head` and `last` take $O(1)$ steps in the worst-case and depending on the instance of the primitive deque, `cons` with `snoc` run in $O(1)$ worst-case or amortized time.

Next two operations, `tail` and `init`, are symmetric in implementation and just operate in $O(1)$ amortized time on either the front or the rear part of either a deque. In case of a `Shallow`, the operation simply propagates to the primitive deque but for a `Deep` deque there are three cases to consider:
 - `tail f` (`init r`) has at lest than 2 elements => new `Deep` can safely be constructed with that part in place
 - there is not enough elements and the middle is empty => the deque downgrades into a `Shallow` one with the remaining elements transferred to the appropriate side of the rest (using either `dappendL` or `dappendR`)
 - the result is too small and the middle is non-empty => appropriate side (primitive deque) is taken from the middle and the remaining elements are transferred to it as in the previous case forming a new front or rear of new `Deep` deque

Finally, the concatenation operator `++` can be proved to run in $O(log(n))$ amortized time and takes an advantage of the `Deque` operations described above. There are 4 cases depending on the operand types involved in `++` but two of them are symmetric:
 - both deques are `Shallow` => if either one is too small (has less than 2 elements) then it's appended to the appropriate side of the other and the result is `Shallow`, otherwise both operands fill the sides of a new `Deep` deque
 - one of the operands is `Shallow` and the other is `Deep` => if the shallow deque is too small then its element is moved to the appropriate side of the deep one, otherwise it replaces such side and the original front/rear is moved to the middle part
 - both deques are `Deep` => front of the former and rear of the latter deque form new front and rear and new middle is constructed by concatenating the rest of the first operand with the initial part of the second operand

Last but not least, the complexity of `++` is indeed logarithmic but it can be stated more precisely with respect to the sizes $n_1$ and $n_2$ of the first and second operand respectively as $O(min(log(n_1), log(n_2)))$.

In [ ]:
data SimpleCatDeque d a = Shallow (d a) | Deep (d a) (SimpleCatDeque d (d a)) (d a)

-- | Determines whether given deque 'd' contains less than 2 elements (i.e. is empty or singleton).
-- |
-- | Note: Deque 'd' is assumed to provide 'tail' in O(1) worst-case or amortized time.
tooSmall :: Deque d => d a -> Bool
tooSmall d = isEmpty d || isEmpty (tail d)

-- | Rotate the head of the first deque to the front of the second one.
-- |
-- | Note: Deque 'd' is assumed to provide 'cons' and 'head' in O(1) worst-case or amortized time.
dappendL :: Deque d => d a -> d a -> d a
dappendL d1 d2 = if isEmpty d1 then d2 else cons (head d1) d2

-- | Rotate the head of the second deque to the back of the first one.
-- |
-- | Note: Deque 'd' is assumed to provide 'snoc' and 'head' in O(1) worst-case or amortized time.
dappendR :: Deque d => d a -> d a -> d a
dappendR d1 d2 = if isEmpty d2 then d1 else snoc d1 (head d2)

-- | 'Deque' instance for 'SimpleCatDeque d' given an arbitrary deque 'd'.
-- |
-- | Assumption: Deque 'd' supports all operations in O(1) time (either amortized or worst-case).
instance Deque d => Deque (SimpleCatDeque d) where
    
    -- | Construct new (empty) deque.
    empty = Shallow empty
    
    -- | Check whether a deque is empty.
    isEmpty (Shallow d) = isEmpty d
    isEmpty _ = False
    
    -- | Prepend an element to the front in O(1) worst-case or amortized time (depends on 'd').
    cons x (Shallow d) = Shallow (cons x d)
    cons x (Deep f m r) = Deep (cons x f) m r
    
    -- | Peek the front element of the deque in O(1) worst case time.
    head (Shallow d) = head d
    head (Deep f _ _) = head f
    
    -- | Remove the front element from a non-empty deque in O(1) amortized time.
    -- |
    -- | Note: By the assumtion on 'd', 'dappendL' runs in constant time (amortized or worst-case).
    tail (Shallow d) = Shallow (tail d)
    tail (Deep f m r)
            | not $ tooSmall f' = Deep f' m r
            | isEmpty m = Shallow (dappendL f' r)
            | otherwise = Deep (dappendL f' (head m)) (tail m) r
        where f' = tail f
    
    -- | Appends an element to the back in O(1) worst-case or amortized time (depends on 'd').
    snoc (Shallow d) x = Shallow (snoc d x)
    snoc (Deep f m r) x = Deep f m (snoc r x)
    
    -- | Peek the rear element of the deque in O(1) worst case time.
    last (Shallow d) = last d
    last (Deep _ _ r) = last r
    
    -- | Remove the rear element from a non-empty deque in O(1) amortized time.
    -- |
    -- | Note: By the assumtion on 'd', 'dappendR' runs in constant time (amortized or worst-case).
    init (Shallow d) = Shallow (init d)
    init (Deep f m r)
            | not $ tooSmall r' = Deep f m r'
            | isEmpty m = Shallow (dappendR f r')
            | otherwise = Deep f (init m) (dappendR (last m) r')
        where r' = init r

instance Deque d => CatenableDeque (SimpleCatDeque d) where
    
    -- | Concatenate two deques together in O(log(n)) amortized time.
    (Shallow d1) ++ (Shallow d2)
        | tooSmall d1 = Shallow (dappendL d1 d2)
        | tooSmall d2 = Shallow (dappendR d1 d2)
        | otherwise = Deep d1 empty d2
    (Shallow d) ++ (Deep f m r)
        | tooSmall d = Deep (dappendL d f) m r
        | otherwise = Deep d (cons f m) r
    (Deep f m r) ++ (Shallow d)
        | tooSmall d = Deep f m (dappendR r d)
        | otherwise = Deep f (snoc m r) d
    (Deep f1 m1 r1) ++ (Deep f2 m2 r2) = Deep f1 (snoc m1 r1 ++ cons f2 m2) r2

## Implicit Catenable Deque
`ImplicitCatDeque` is a *structural abstraction* of standard `Deque`s that implements the *implicit recursive slowdown* to provide a concatenation operator that runs in $O(1)$ amortized time.

It is similar to the `SimpleCatDeque` but with more segments and more complicated *compound elements* held by the recursive parts.

### Representation
The `ImplicitCatDeque` is similar in structure to the `SimpleCatDeque` but is composed of five segments `(f, a, m, b, r)` instead of three:
 - the front part `f` is an ordinary `Deque` that must contain at least 3 elements
 - a catenable deque `a` of *compound elements*
 - the middle `m` is now an ordinary `Deque` that must contain at least 2 elements
 - a catenable deque `b` of *compound elements*
 - the rear part `r` is an ordinary `Deque` that must contain at least 3 elements

A *compound element* can either be:
 - `Simple` - holding an ordinary `Deque` of at least two elements
 - `Cmpd` - a compound element `(f, c, r)` where `f` and `r` are ordinary `Deque`s (again of size at least 2) and `c` is a catenable deque of compound elements

Note that the definition of `ImplicitCatDeque` described above assumes language support for *polymorphic recursion*.

### Operations
All the "basic" operation like `cons`, `snoc`, `head` and `tail` are analogous in implementation to the ones of `SimpleCatDeque` and run in $O(1)$ worst-case or amortized time by the assumtion on the underlying `Deque`.

Unfortunately, `tail` and `init` (which is again symmetrical to `tail`) now have more cases to consider. In general they remove the primitive front element and reconstruct the rest of the queue by moving parts around while maintining the invariant on indiviual segments.

For instance for the `tail` function it first depends whether 1) the front deque has more than 3 elements (i.e. enought to just remove the head) or 2) is of the size 3, just barely satisfying the invariant. In the latter case there are three further possibilities:
 1. segment `a` is non-empty and can be borrowed from to reconstruct valit rear
 1. segment `a` is empty but `b` is non-empty and again deques (elements) can be moved from it to the middle and rear
 1. both compound element deques `a` and `b` are empty => remaining pards are downgraded to `Shallow` deques and concatenated to form a new one

It can be shown (see the book) that both `tail` and `init` run in $O(1)$ amortized time.

Finally, the idea behind improving the complexity of `++` operator is to make it not call itself recursively. This means that `++` can only call `cons` and `snoc` at the next level (which are by the deque assumption constant operations).

Similarly to the concatenation of `SimpleCatDeque`, the implementation here has four cases for each combination of `Shallow` and `Deep` deques but in general it is just shuffeling segments and elements around and since all the basic operations are constant, `++` can be shown to run in $O(1)$ amortized time.

In [ ]:
data ImplicitCatDeque d a = Shallow (d a)
                          | Deep (d a) -- size >= 3
                                 (ImplicitCatDeque d (CmpdElem d a))
                                 (d a) -- size >= 2
                                 (ImplicitCatDeque d (CmpdElem d a))
                                 (d a) -- size >= 3

data CmpdElem d a = Simple (d a) -- size >= 2
                  | Cmpd (d a)   -- size >= 2
                         (ImplicitCatDeque d (CmpdElem d a))
                         (d a)   -- size >= 2

-- | Make a new 'Deque' from the rear element of the first deque and the front
-- | element of the second deque.
-- |
-- | Since 'cons', 'head' and 'last' from 'd' are assumed to run in O(1) time,
-- | so does 'share'.
share :: Deque d => d a -> d a -> (d a, d a, d a)
share f r = (init f, m, tail r) where m = cons (last f) (cons (head r) empty)

-- | Rotate the the first deque to the front of the second one.
dappendL :: Deque d => d a -> d a -> d a
dappendL d1 d2 =
  if isEmpty d1 then d2 else dappendL (init d1) (cons (last d1) d2)

-- | Rotate the the second deque to the back of the first one.
dappendR :: Deque d => d a -> d a -> d a
dappendR d1 d2 =
  if isEmpty d2 then d1 else dappendR (snoc d1 (head d2)) (tail d2)

-- | Efficiently replace the head of a 'ImplicitCatDeque' with given element.
-- |
-- | Since this implemntation uses just 'cons' and 'tail' from the underlying
-- | deque 'd', it's running time is O(1) (worst-case or amortized).
replaceHead :: Deque d => a -> ImplicitCatDeque d a -> ImplicitCatDeque d a
replaceHead x (Shallow d     ) = Shallow (cons x (tail d))
replaceHead x (Deep f a m b r) = Deep (cons x (tail f)) a m b r

instance (Deque d, Sized d) => Deque (ImplicitCatDeque d) where

    -- | Construct new (empty) deque.
  empty = Shallow empty

  -- | Check whether a deque is empty.
  isEmpty (Shallow d) = isEmpty d
  isEmpty _           = False

  -- | Prepend an element to the front in O(1) worst-case or amortized time (depends on 'd').
  cons x (Shallow d     ) = Shallow (cons x d)
  cons x (Deep f a m b r) = Deep (cons x f) a m b r

  -- | Peek the front element of the deque in O(1) worst case time.
  head (Shallow d     ) = head d
  head (Deep f _ _ _ _) = head f

  -- | Remove the front element from a non-empty deque in O(1) amortized time.
  tail (Shallow d) = Shallow (tail d)
  tail (Deep f a m b r)
    |
    -- 'f' has enough elements in to pop one out and preserve the size invariant
      size f > 3 = Deep (tail f) a m b r
    |
    -- the front is too small but 'a' is not empty and can be borrowed from
      not $ isEmpty a = case head a of
      -- head of 'a' is a simple deque => we can append the rest of 'f' to it
      Simple d      -> Deep f' (tail a) m b r where f' = dappendL (tail f) d
      -- head of 'a' is compound => we steal it's front and rectonstruct 'a' by
      -- replacing it's head with the compound's rear and prepending the center
      Cmpd f' c' r' -> Deep f'' a'' m b r
       where
        f'' = dappendL (tail f) f'
        a'' = c' ++ replaceHead (Simple r') a
    |
    -- 'a' is empty but 'b' is not
      not $ isEmpty b = case head b of
      -- head of 'b' is a simple deque => we leave it in place of 'm' which is
      -- appended to the rest of 'f'
      Simple d      -> Deep f' empty d (tail b) r where f' = dappendL (tail f) m
      -- head of 'b' is compound => we steal the middle 'm' for the new front
      -- and replace it with the compound's rear; front and center make new 'a'
      Cmpd f' c' r' -> Deep f'' a'' r' (tail b) r
       where
        f'' = dappendL (tail f) m
        a'' = cons (Simple f') c'
    |
    -- both 'a' and 'b' deques are empty => combine remaining front, 'm' and 'r'
      otherwise = Shallow (dappendL (tail f) m) ++ Shallow r

  -- | Appends an element to the back in O(1) worst-case or amortized time (depends on 'd').
  snoc (Shallow d     ) x = Shallow (snoc d x)
  snoc (Deep f a m b r) x = Deep f a m b (snoc r x)

  -- | Peek the rear element of the deque in O(1) worst case time.
  last (Shallow d     ) = last d
  last (Deep _ _ _ _ r) = last r

  -- | Remove the rear element from a non-empty deque in O(1) amortized time.
  init (Shallow d) = Shallow (init d)
  init (Deep f a m b r)
    |
    -- 'r' has enough elements in to pop one out and preserve the size invariant
      size r > 3 = Deep f a m b (init r)
    |
    -- the rear is too small but 'b' is not empty and can be borrowed from
      not $ isEmpty b = case last b of
      -- last of 'b' is a simple deque => we can append the init of 'r' to it
      Simple d      -> Deep f a m (init b) r' where r' = dappendR d (init r)
      -- last of 'b' is compound => we steal it's rear and rectonstruct 'b' by
      -- prepending compound's front to its center
      Cmpd f' c' r' -> Deep f a m b'' r''
       where
        r'' = dappendR r' (init r)
        b'' = cons (Simple f') c'
    |
    -- 'b' is empty but 'a' is not
      not $ isEmpty a = case last a of
      -- last of 'a' is a simple deque => we leave it in place of 'm' which is
      -- prepended to the rest of 'r'
      Simple d      -> Deep f (init a) d empty r' where r' = dappendR m (init r)
      -- last of 'a' is compound => we steal the middle 'm' for the new rear and
      -- replace it with the compound's front; center and rear make new 'b'
      Cmpd f' c' r' -> Deep f (init a) f' b'' r''
       where
        r'' = dappendR m (init r)
        b'' = snoc c' (Simple r')
    |
    -- both 'b' and 'a' deques are empty => combine remaining rear, 'm' and 'f'
      otherwise = Shallow f ++ Shallow (dappendR m (init r))

instance (Deque d, Sized d) => CatenableDeque (ImplicitCatDeque d) where

  -- | Concatenate two deques together in O(1) amortized time.
  (Shallow d1) ++ (Shallow d2)
    | size d1 < 4 = Shallow (dappendL d1 d2)
    | size d2 < 4 = Shallow (dappendR d1 d2)
    | otherwise   = let (f, m, r) = share d1 d2 in Deep f empty m empty r
  (Shallow d) ++ (Deep f a m b r)
    | size d < 4 = Deep (dappendL d f) a m b r
    | otherwise  = Deep d (cons (Simple f) a) m b r
  (Deep f a m b r) ++ (Shallow d)
    | size d < 4 = Deep f a m b (dappendR r d)
    | otherwise  = Deep f a m (snoc b (Simple r)) d
  (Deep f1 a1 m1 b1 r1) ++ (Deep f2 a2 m2 b2 r2) = Deep f1 a1' m b2' r2
   where
    -- make new middle 'm' from the last and front elements of 'r1' and 'f2'
    (r1', m, f2') = share r1 f2
    -- make new 'a' from 'a1' with the rest of the first deque combined
    a1'           = snoc a1 (Cmpd m1 b1 r1')
    -- make new 'b' from 'b2' and the initial part of the second deque combined
    b2'           = cons (Cmpd f2' a2 m2) b2